In [ ]:
import json
import os
import logging
import pandas as pd
from jira import JIRA
import dash
from dash import dcc, html, dash_table
import plotly.express as px
from datetime import datetime

In [ ]:
# Função para configurar logging dinamicamente
def setup_logging(log_file_path):
    # Cria o handler e o formatter para o logging
    handler = logging.FileHandler(log_file_path)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    
    # Configura o logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)  # Nível de log
    logger.addHandler(handler)
    
    return logger


In [ ]:
# Função para carregar configurações do arquivo JSON
def load_config(config_file='configs/jira_configs.json'):
    if not os.path.exists(config_file):
        print(f"Arquivo de configuração '{config_file}' não encontrado. Criando arquivo com configurações padrão.")
        # Garantir que o diretório existe
        os.makedirs(os.path.dirname(config_file), exist_ok=True)
        default_config = {
            "JIRA_SERVER": "https://your_jira_server",
            "JIRA_USER": "your_username",
            "JIRA_TOKEN": "your_token",
            "PROJECT_KEY": "your_project_key",
            "FOLDER_LOGS": ".",
            "FOLFER_REPORTS": "."
        }
        with open(config_file, 'w') as file:
            json.dump(default_config, file, indent=4)
        return default_config
    try:
        with open(config_file, 'r') as file:
            return json.load(file)
    except json.JSONDecodeError:
        print(f"Erro ao ler o arquivo de configuração '{config_file}'. Verifique o formato JSON.")
        raise

# Carregar configurações do arquivo JSON
import os
config = load_config(config_file='configs/jira_configs.json')

In [ ]:
# Conexão com o Jira
options = {'server': config['JIRA_SERVER']}
jira = JIRA(options, basic_auth=(config['JIRA_USER'], config['JIRA_TOKEN']))

In [ ]:
# Função para buscar EPICs e cards do Jira
def buscar_epics_e_cards(jira, project_key):
    # Buscar EPICs
    epics = jira.search_issues(f'project = {project_key} AND issuetype = Epic', maxResults=1000)
    # Buscar cards (tarefas)
    cards = jira.search_issues(f'project = {project_key} AND issuetype = Task', maxResults=1000)

    # Listas para armazenar os dados
    dados_epics = []
    dados_cards = []

    # Extrair informações dos EPICs
    for epic in epics:
        dados_epics.append({
            'Chave': epic.key,
            'Resumo': epic.fields.summary,
            'Status': epic.fields.status.name,
            'Responsável': epic.fields.assignee.displayName if epic.fields.assignee else 'Não atribuído',
            'Data de Criação': epic.fields.created,
            'Data de Atualização': epic.fields.updated
        })

    # Extrair informações dos cards
    for card in cards:
        dados_cards.append({
            'Chave': card.key,
            'Resumo': card.fields.summary,
            'Status': card.fields.status.name,
            'Responsável': card.fields.assignee.displayName if card.fields.assignee else 'Não atribuído',
            'EPIC': card.fields.parent.key if hasattr(card.fields, 'parent') and card.fields.parent else 'Sem EPIC',
            'Data de Criação': card.fields.created,
            'Data de Atualização': card.fields.updated
        })

    # Converter listas em DataFrames
    df_epics = pd.DataFrame(dados_epics)
    df_cards = pd.DataFrame(dados_cards)

    return df_epics, df_cards

In [ ]:
# Função principal
def main():
 
    # Atribuição de variáveis de configuração
    JIRA_SERVER = config['JIRA_SERVER']
    JIRA_USER = config['JIRA_USER']
    JIRA_TOKEN = config['JIRA_TOKEN']
    PROJECT_KEY = config['PROJECT_KEY']
    FOLDER_LOGS = config['FOLDER_LOGS']
    FOLDER_REPORTS = config['FOLDER_REPORTS']

    # Gerar nome dinâmico para o arquivo de log com timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    log_dir = os.path.dirname(FOLDER_LOGS)
    log_file_path = os.path.join(log_dir, f"jira_extract_log_{timestamp}.log")

    # Configuração de logging
    logger = setup_logging(log_file_path)
    logger.info(f"Log gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Buscar EPICs e cards
    try:
        df_epics, df_cards = buscar_epics_e_cards(jira, PROJECT_KEY)
        logger.info("Dados de EPICs e cards extraídos com sucesso.")
    except Exception as e:
        logger.error(f"Erro ao buscar EPICs e cards: {str(e)}")
        raise   

    # Exportar EPICs e cards para CSV
    try:
        epics_csv_path = os.path.join(FOLDER_REPORTS, "epics.csv")
        df_epics.to_csv(epics_csv_path, index=False)
        logger.info(f"EPICs exportados para: {epics_csv_path}")

        cards_csv_path = os.path.join(FOLDER_REPORTS, "cards.csv")
        df_cards.to_csv(cards_csv_path, index=False)
        logger.info(f"Cards exportados para: {cards_csv_path}")
    except Exception as e:
        logger.error(f"Erro ao exportar dados para CSV: {str(e)}")
        raise

    logger.info("Processo de exportação concluído com sucesso.")

    # Criar dashboard com Dash
    app = dash.Dash(__name__)

    # Layout do dashboard
    app.layout = html.Div(children=[
        html.H1(children='Dashboard de EPICs e Cards do Jira'),

        # Gráfico 1: Distribuição de EPICs por status
        dcc.Graph(
            id='epics-status-chart',
            figure=px.bar(df_epics['Status'].value_counts(), 
                          x=df_epics['Status'].value_counts().index, 
                          y=df_epics['Status'].value_counts().values,
                          labels={'x': 'Status', 'y': 'Número de EPICs'},
                          title='Distribuição de EPICs por Status')
        ),

        # Gráfico 2: Distribuição de cards por status
        dcc.Graph(
            id='cards-status-chart',
            figure=px.bar(df_cards['Status'].value_counts(), 
                          x=df_cards['Status'].value_counts().index, 
                          y=df_cards['Status'].value_counts().values,
                          labels={'x': 'Status', 'y': 'Número de Cards'},
                          title='Distribuição de Cards por Status')
        ),

        # Tabela de EPICs
        html.H3(children='Lista de EPICs'),
        dash_table.DataTable(
            id='epics-table',
            columns=[{'name': col, 'id': col} for col in df_epics.columns],
            data=df_epics.to_dict('records'),
            page_size=10
        ),

        # Tabela de Cards
        html.H3(children='Lista de Cards'),
        dash_table.DataTable(
            id='cards-table',
            columns=[{'name': col, 'id': col} for col in df_cards.columns],
            data=df_cards.to_dict('records'),
            page_size=10
        )
    ])

    # Executar o dashboard
    app.run(debug=True)

In [ ]:
# Executar o script
if __name__ == "__main__":
    main()